# Homework 2

Let's create a social media account for your agent

# Setup your agent

In [75]:

# 📦 Install Required Packages
!pip install langchain-google-genai langchain-core langchain-experimental
!pip install yfinance
!pip install tenacity --quiet

In [76]:

# 🔑 API Key Setup
from google.colab import userdata
GEMINI_VERTEX_API_KEY = userdata.get('VERTEX_API_KEY')
assert GEMINI_VERTEX_API_KEY, "Please set your VERTEX_API_KEY in Colab secrets"

In [77]:

# 🤖 Initialize Gemini LLM
from langchain_google_genai import ChatGoogleGenerativeAI

llm = ChatGoogleGenerativeAI(
    model="gemini-2.5-flash",
    api_key=GEMINI_VERTEX_API_KEY,
    vertexai=True,
    temperature=0
)

# Create a moltbook account for your agent

In [78]:
# This function is used to encode your student id to ensure the privacy

def encode_student_id(student_id: int) -> str:
    """
    Reversibly encode a student ID using an affine cipher.

    Args:
        student_id (int): Original student ID (non-negative integer)

    Returns:
        str: Encoded ID as a zero-padded string
    """
    if student_id < 0:
        raise ValueError("student_id must be non-negative")

    M = 10**8
    a = 137
    b = 911

    encoded = (a * student_id + b) % M
    return f"{encoded:08d}"

In [79]:
# Before creating your agent please encode your student id using this function and replace XXXX by the encoded number
encode_student_id(32463728)

'47531647'

In [80]:
# Please use the encoded student id
!curl -X POST https://www.moltbook.com/api/v1/agents/register \
  -H "Content-Type: application/json" \
  -d '{"name": "BaoNguyen_367126387", "description": "Va"}'

{"statusCode":429,"message":"Rate limit exceeded","remaining":0,"reset_at":"2026-02-26T07:35:03.230Z","retry_after_seconds":84358,"timestamp":"2026-02-25T08:09:05.230Z","path":"/api/v1/agents/register"}

- After sucessfully register, you will see a notification of the format:

"success":true,"message":"Welcome to Moltbook! 🦞","agent":"id":"...","name":"...","api_key":"...", "claim_url": "..."

- Please save your the api key as MOLTBOOK_API_KEY in the Secrets section of your Colab.
- Then you complete the registration by accessing the claim_url and follow the guideline in the url.

In [81]:
# Create a tool set to interact with moltbook

import os
import requests
from langchain_core.tools import tool
from tenacity import (
    retry,
    stop_after_attempt,
    wait_exponential,
    retry_if_exception_type
)
MOLTBOOK_API_KEY = userdata.get('MOLTBOOK_API_KEY')
BASE_URL = "https://www.moltbook.com/api/v1"

HEADERS = {
    "Authorization": f"Bearer {MOLTBOOK_API_KEY}",
    "Content-Type": "application/json"
}

# ---------- Core Moltbook Tools ----------
@tool
def get_feed(sort: str = "new", limit: int = 10) -> dict:
    """Fetch Moltbook feed."""
    r = requests.get(
        f"{BASE_URL}/feed",
        headers=HEADERS,
        params={"sort": sort, "limit": limit},
        timeout=15
    )
    return r.json()

@tool
def search_moltbook(query: str, type: str = "all") -> dict:
    """Semantic search Moltbook posts, comments, agents."""
    r = requests.get(
        f"{BASE_URL}/search",
        headers=HEADERS,
        params={"q": query, "type": type},
        timeout=15
    )
    return r.json()

@tool
def create_post(submolt: str, title: str, content: str) -> dict:
    """Create a new text post."""
    payload = {
        "submolt": submolt,
        "title": title,
        "content": content
    }
    r = requests.post(
        f"{BASE_URL}/posts",
        headers=HEADERS,
        json=payload,
        timeout=15
    )
    return r.json()

@tool
def comment_post(post_id: str, content: str) -> dict:
    """Comment on a post."""
    r = requests.post(
        f"{BASE_URL}/posts/{post_id}/comments",
        headers=HEADERS,
        json={"content": content},
        timeout=15
    )
    return r.json()

@tool
def upvote_post(post_id: str) -> dict:
    """Upvote a post."""
    r = requests.post(
        f"{BASE_URL}/posts/{post_id}/upvote",
        headers=HEADERS,
        timeout=15
    )
    return r.json()

# ---------- Additional Required Tools ----------
@tool
@retry(
    stop=stop_after_attempt(3),
    wait=wait_exponential(multiplier=1, min=2, max=10),
    retry=retry_if_exception_type((requests.exceptions.RequestException, requests.exceptions.Timeout))
)
def get_submolt(submolt_name: str) -> dict:
    """Retrieve details of a specific submolt by name"""
    r = requests.get(
        f"{BASE_URL}/submolts/{submolt_name}",
        headers=HEADERS,
        timeout=15
    )
    r.raise_for_status()
    return r.json()

@tool
@retry(
    stop=stop_after_attempt(3),
    wait=wait_exponential(multiplier=1, min=2, max=10),
    retry=retry_if_exception_type((requests.exceptions.RequestException, requests.exceptions.Timeout))
)
def get_submolt_posts(submolt_name: str, sort: str = "new", limit: int = 10) -> dict:
    """Get posts from a specific submolt"""
    r = requests.get(
        f"{BASE_URL}/submolts/{submolt_name}/posts",
        headers=HEADERS,
        params={"sort": sort, "limit": limit},
        timeout=15
    )
    r.raise_for_status()
    return r.json()

@tool
@retry(
    stop=stop_after_attempt(3),
    wait=wait_exponential(multiplier=1, min=2, max=10),
    retry=retry_if_exception_type((requests.exceptions.RequestException, requests.exceptions.Timeout))
)
def subscribe_submolt(submolt_name: str) -> dict:
    """Subscribe to a specific submolt"""
    r = requests.post(
        f"{BASE_URL}/submolts/{submolt_name}/subscribe",
        headers=HEADERS,
        timeout=15
    )
    r.raise_for_status()
    return r.json()

@tool
@retry(
    stop=stop_after_attempt(3),
    wait=wait_exponential(multiplier=1, min=2, max=10),
    retry=retry_if_exception_type((requests.exceptions.RequestException, requests.exceptions.Timeout))
)
def get_agent_status() -> dict:
    """Check current agent status (claimed/active)"""
    r = requests.get(
        f"{BASE_URL}/agents/status",
        headers=HEADERS,
        timeout=15
    )
    r.raise_for_status()
    return r.json()

# ---------- Validation Helper Functions ----------
def check_duplicate_content(submolt: str, content: str) -> bool:
    """Check if content already exists in the target submolt to avoid duplication"""
    search_result = search_moltbook.invoke({
        "query": content[:50],
        "type": "posts",
    })
    if "posts" in search_result and len(search_result["posts"]) > 0:
        for post in search_result["posts"]:
            if post.get("submolt") == submolt and (content in post.get("content", "") or post.get("title") in content):
                return True
    return False

def has_new_insight(comment_content: str, post_content: str) -> bool:
    """Verify comment adds new insight (not just repeating post content)"""
    post_words = set(post_content.lower().split())
    comment_words = set(comment_content.lower().split())
    overlap = len(post_words & comment_words) / len(comment_words) if comment_words else 1.0
    new_insight_keywords = ["agentic system", "langchain", "gemini", "prompt engineering", "FTEC5660"]
    has_new_keyword = any(word in comment_content.lower() for word in new_insight_keywords)
    return overlap < 0.5 and has_new_keyword

In [82]:
# ---------- System Prompt ----------
SYSTEM_PROMPT = """
You are a Moltbook AI agent.

Your purpose:
- Discover valuable AI / ML / agentic system discussions
- Engage thoughtfully and selectively
- NEVER spam
- NEVER repeat content
- Respect rate limits

Rules:
1. Before posting, ALWAYS search Moltbook to avoid duplication.
2. Only comment if you add new insight.
3. Upvote only genuinely useful content.
4. If uncertain, do nothing.
5. Prefer short, clear, professional language.
6. If a human gives an instruction, obey it exactly.

Available tools:
- get_feed
- search_moltbook
- create_post
- comment_post
- upvote_post
- get_submolt
- get_submolt_posts
- subscribe_submolt
- get_agent_status
"""

# A simple agent to interact with moltbook

In [83]:
import datetime
from google.colab import userdata
import requests
import json
import time
import random
from tenacity import retry, stop_after_attempt, wait_exponential, retry_if_exception_type
from langchain_core.tools import tool
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_core.messages import ToolMessage

def log(section: str, message: str):
    ts = datetime.datetime.now(datetime.UTC).strftime("%H:%M:%S")
    print(f"[{ts}] [{section}] {message}")

def pretty(obj: Any, max_len: int = 800):
    text = json.dumps(obj, indent=2, ensure_ascii=False, default=str)
    return text if len(text) <= max_len else text[:max_len] + "\n...<truncated>"

# ---------- Core Agent Loop ----------
def moltbook_agent_loop(
    instruction: str | None = None,
    max_turns: int = 12,
    verbose: bool = True,
):
    log("INIT", "Starting Moltbook agent loop")

    # Initialize LLM
    llm = ChatGoogleGenerativeAI(
        model="gemini-2.5-flash",
        temperature=0,
        api_key=GEMINI_VERTEX_API_KEY,
        vertexai=True,
    )

    # Assemble all tools
    tools = [
        get_feed,
        search_moltbook,
        create_post,
        comment_post,
        upvote_post,
        get_submolt,
        get_submolt_posts,
        subscribe_submolt,
        get_agent_status
    ]
    agent = llm.bind_tools(tools)

    # Initialize conversation history
    history = [("system", SYSTEM_PROMPT)]

    # Check agent status first
    try:
        status_result = get_agent_status.invoke({})
        if not status_result.get("success") or status_result.get("status") != "claimed":
            log("ERROR", f"Agent status invalid: {status_result}")
            return f"Agent initialization failed: {status_result}"
        log("STATUS", "Agent status is claimed (valid) ✅")
    except Exception as e:
        log("ERROR", f"Failed to check agent status: {str(e)}")
        return f"Status check failed: {str(e)}"

    # Process user instruction
    if instruction:
        history.append(("human", f"Human instruction: {instruction}"))
        log("HUMAN", instruction)
    else:
        history.append(("human", "Perform your Moltbook heartbeat check."))
        log("HEARTBEAT", "No human instruction – autonomous mode")

    # Main agent execution loop
    for turn in range(1, max_turns + 1):
        log("TURN", f"Turn {turn}/{max_turns} started")
        turn_start = time.time()

        # Get LLM response
        response = agent.invoke(history)
        history.append(response)

        if verbose:
            log("LLM", "Model responded")
            log("LLM.CONTENT", response.content or "<empty>")
            log("LLM.TOOL_CALLS", pretty(response.tool_calls or []))

        # Stop if no tool calls
        if not response.tool_calls:
            elapsed = round(time.time() - turn_start, 2)
            log("STOP", f"No tool calls — final answer produced in {elapsed}s")
            return response.content

        # Execute tool calls
        for i, call in enumerate(response.tool_calls, start=1):
            tool_name = call["name"]
            args = call["args"]
            tool_id = call["id"]

            log("TOOL", f"[{i}] Calling `{tool_name}`")
            log("TOOL.ARGS", pretty(args))

            # Get tool function
            tool_fn = globals().get(tool_name)
            if not tool_fn:
                result = {"error": f"Tool {tool_name} not found"}
                status = "error"
                tool_elapsed = 0
            else:
                tool_start = time.time()
                try:
                    # Content validation for posts/comments
                    if tool_name == "create_post":
                        submolt = args.get("submolt")
                        content = args.get("content")
                        if check_duplicate_content(submolt, content):
                            result = {"error": "Content duplicate, skip post creation"}
                            status = "skipped"
                        else:
                            result = tool_fn.invoke(args)
                            status = "success"
                    elif tool_name == "comment_post":
                        post_id = args.get("post_id")
                        comment_content = args.get("content")
                        # Get post content for insight check
                        post_detail = requests.get(
                            f"{BASE_URL}/posts/{post_id}",
                            headers=HEADERS,
                            timeout=15
                        ).json()
                        post_content = post_detail.get("content", "")
                        if not has_new_insight(comment_content, post_content):
                            result = {"error": "No new insight, skip comment"}
                            status = "skipped"
                        else:
                            result = tool_fn.invoke(args)
                            status = "success"
                    else:
                        # Execute other tools directly
                        result = tool_fn.invoke(args)
                        status = "success"
                except Exception as e:
                    result = {"error": str(e)}
                    status = "error"
                tool_elapsed = round(time.time() - tool_start, 2)

            # Log tool result
            log("TOOL.RESULT", f"{tool_name} finished ({status}) in {tool_elapsed}s")
            if verbose:
                log("TOOL.OUTPUT", pretty(result))

            # Add tool result to history
            history.append(
                ToolMessage(
                    tool_call_id=tool_id,
                    content=str(result),
                )
            )

            # Rate limiting
            time.sleep(random.uniform(1, 2))

        turn_elapsed = round(time.time() - turn_start, 2)
        log("TURN", f"Turn {turn} completed in {turn_elapsed}s")

    # Stop if max turns reached
    log("STOP", "Max turns reached without final answer")
    return "Agent stopped after reaching max turns."


In [84]:
# You need to complte the tool set so that your agent can find the submolt
# moltbook_agent_loop("find submolt named ftec5660")

# ---------- Full Task Execution ----------
full_instruction = """
Execute these steps in order:
1. Use the `get_submolt` tool to find the submolt named ftec5660 (confirm its name and ID);
2. Use the `subscribe_submolt` tool to subscribe to the ftec5660 submolt;
3. Upvote the post with ID: 47ff50f3-8255-4dee-87f4-2c3637c7351c (use upvote_post tool);
4. Comment on this post with a professional insight about AI/agentic systems (1-2 sentences, add real value, no spam).
"""

# Run the agent
final_result = moltbook_agent_loop(instruction=full_instruction, max_turns=12)
print("Final Execution Result: ", final_result)

[08:09:05] [INIT] Starting Moltbook agent loop
[08:09:06] [STATUS] Agent status is claimed (valid) ✅
[08:09:06] [HUMAN] 
Execute these steps in order:
1. Use the `get_submolt` tool to find the submolt named ftec5660 (confirm its name and ID);
2. Use the `subscribe_submolt` tool to subscribe to the ftec5660 submolt;
3. Upvote the post with ID: 47ff50f3-8255-4dee-87f4-2c3637c7351c (use upvote_post tool);
4. Comment on this post with a professional insight about AI/agentic systems (1-2 sentences, add real value, no spam).

[08:09:06] [TURN] Turn 1/12 started
[08:09:09] [LLM] Model responded
[08:09:09] [LLM.CONTENT] <empty>
[08:09:09] [LLM.TOOL_CALLS] [
  {
    "name": "get_submolt",
    "args": {
      "submolt_name": "ftec5660"
    },
    "id": "618571fb-9a1b-4488-bb04-685835892060",
    "type": "tool_call"
  }
]
[08:09:09] [TOOL] [1] Calling `get_submolt`
[08:09:09] [TOOL.ARGS] {
  "submolt_name": "ftec5660"
}
[08:09:09] [TOOL.RESULT] get_submolt finished (success) in 0.36s
[08:09:09] [